**Objective:** 
Your task is to program an agent to find the optimal policy for navigating a labyrinth from a specified starting point to a goal point using the Value Iteration algorithm.

**Step 1: Familiarize with the Environment**
- Understand the structure of the `Labyrinth` class and how it represents the labyrinth environment, including walls, the starting point, and the goal.
- Familiarize yourself with how the `Agent` class is structured, and how it interacts with the labyrinth environment.

**Step 2: Implement Value Iteration**
- Create a function or method to implement the Value Iteration algorithm.
- You'll need to initialize a utility table with zeros and iteratively update the utilities of each state (i.e., each cell in the labyrinth) based on the Bellman equation.
- The stopping criterion for Value Iteration is when the maximum change in utility is less than a small threshold, say 0.01.
- Once the utilities have converged, use them to compute the optimal policy, which specifies the best action to take in each state.

**Step 3: Modify the Agent Class**
- Modify the `act` method of the `Agent` class to use the optimal policy derived from Value Iteration instead of taking random actions.
- Optionally, you can also modify the `update` method to incorporate any additional learning or updating you wish to implement.

**Step 4: Run the Simulation**
- Run the provided simulation loop, where the agent is placed in the labyrinth and must navigate to the goal.
- Observe how the agent's behavior changes as it learns the optimal policy.
- You might want to add some print statements or other logging to help visualize the agent's path through the labyrinth and how it improves over time.



In [30]:
import numpy as np
import random 
from time import sleep

class Labyrinth:
    def __init__(self, rows, cols, walls, start, goal):
        self.walls = walls
        self.grid = np.zeros((rows, cols))
        for wall in walls:
            self.grid[wall] = -1  # Assign -1 for walls
        self.start = start
        self.goal = goal
        self.current_position = start

    def reset(self):
        self.current_position = self.start
        return self.current_position

    def render(self, epoch=0, t=0, sleep_time=1):
        print(f'epoch: {epoch}, t: {t}')
        grid_copy = self.grid.copy()
        grid_copy[self.current_position] = 2
        grid_copy[self.goal] = 9
        display(grid_copy)
        print('-' * 50)
        sleep(sleep_time)

    def step(self, action: tuple[int, int]):
        # Assume actions are encoded as (delta_row, delta_col)
        new_position = (self.current_position[0] + action[0], self.current_position[1] + action[1])
        if self.is_valid_move(new_position):
            self.current_position = new_position
        reward = 1 if self.current_position == self.goal else 0
        return self.current_position, reward

    def is_valid_move(self, position):
        rows, cols = self.grid.shape
        return 0 <= position[0] < rows and 0 <= position[1] < cols and self.grid[position] != -1

    def done(self):
        return self.current_position == self.goal


class Action:
    def __init__(self, action, state):
        self.action = action
        self.state = state


class Agent:
    def __init__(self, lab: Labyrinth, gamma=0.5):
        self.lab = lab
        self.utility_table = np.zeros(lab.grid.shape)

        for wall in lab.walls:
            self.utility_table[wall] = -1

        self.gamma = gamma
        self.actions = {
            'left': (0, -1),
            'right': (0, 1),
            'up': (-1, 0),
            'down': (1, 0)
        }

    def reset(self):
        pass

    def act(self, state):
        next_best_state = self.select_best_action(state).action
        best_action = self.actions[next_best_state]
        # return random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])  # Random action for demonstration
        return best_action

    def max_a_utility(self, state: tuple[int, int]):
        return max([self.utility_table[s.state] for s in self.potential_states(state)])

    def select_best_action(self, state) -> Action:
        return max(self.potential_states(state), key=lambda a: self.utility_table[a.state])

    def potential_states(self, state):
        potential = []
        rows, cols = self.utility_table.shape

        for a in self.actions:
            s_new_y, s_new_x = self.actions[a]
            s_new_y += state[0]
            s_new_x += state[1]

            if (0 <= s_new_y < rows and 0 <= s_new_x < cols) and (s_new_y, s_new_x) not in self.lab.walls:
                potential.append(
                    Action(a, (s_new_y, s_new_x))
                )

        return potential

    def update_utility_table(self):
        new_utility_table = np.copy(self.utility_table)

        for y in range(self.utility_table.shape[0]):
            for x in range(self.utility_table.shape[1]):
                s = (y, x)
                if s not in self.lab.walls:
                    new_utility_table[s] = self.bellman_equation(s)

        self.utility_table = new_utility_table

    def bellman_equation(self, s):
        return self.get_reward(s) + self.gamma * self.max_a_utility(s)

    def update(self, action, state, reward):
        pass

    def get_reward(self, state):
        if state == self.lab.goal:
            return 1
        elif state in self.lab.walls:
            return -1
        else:
            return 0


In [31]:
# Define labyrinth
labyrinth = Labyrinth(4, 4, {(1, 1), (2, 1), (1, 2)}, (0, 0), (3, 3))
agent = Agent(lab=labyrinth, gamma=0.5)

MAX_EPISODES = 1000
T = 100

for episode in range(MAX_EPISODES):
    state = labyrinth.reset()
    agent.reset()

    for t in range(T):
        agent.update_utility_table()
        action = agent.act(state)
        state, reward = labyrinth.step(action)
        if labyrinth.done():
            break
agent.utility_table

array([[ 0.02083333,  0.04166667,  0.08333333,  0.16666667],
       [ 0.04166667, -1.        , -1.        ,  0.33333333],
       [ 0.08333333, -1.        ,  0.33333333,  0.66666667],
       [ 0.16666667,  0.33333333,  0.66666667,  1.33333333]])

In [32]:
# test agent
state = labyrinth.reset()
agent.reset()

print("let's get it! 2 is agent, 9 is goal")
labyrinth.render()

for t in range(T):
    action = agent.act(state)
    state, reward = labyrinth.step(action)
    
    labyrinth.render(epoch=-1, t=t, sleep_time=0.5)
    if labyrinth.done():
        print("Goal reached!\n")
        break

let's get it! 2 is agent, 9 is goal
epoch: 0, t: 0


array([[ 2.,  0.,  0.,  0.],
       [ 0., -1., -1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  9.]])

--------------------------------------------------
epoch: -1, t: 0


array([[ 0.,  2.,  0.,  0.],
       [ 0., -1., -1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  9.]])

--------------------------------------------------
epoch: -1, t: 1


...

--------------------------------------------------
epoch: -1, t: 4


array([[ 0.,  0.,  0.,  0.],
       [ 0., -1., -1.,  0.],
       [ 0., -1.,  0.,  2.],
       [ 0.,  0.,  0.,  9.]])

--------------------------------------------------
epoch: -1, t: 5


array([[ 0.,  0.,  0.,  0.],
       [ 0., -1., -1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  9.]])

--------------------------------------------------
Goal reached!

